<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a Haystack Application with Phoenix</h1>

ℹ️ This notebook requires an OpenAI API key.


In [1]:
!pip install -q openinference-instrumentation-haystack haystack-ai arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp arize-phoenix-otel


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


# Set API Keys

In [ ]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["PHOENIX_API_KEY"] = getpass.getpass("Phoenix API Key:")

OpenAI API Key:··········
Phoenix API Key:··········


# Launch Phoenix and Enable Haystack Tracing

In [ ]:
import os
from openinference.instrumentation.haystack import HaystackInstrumentor
from phoenix.otel import register

PHOENIX_API_KEY = os.environ["PHOENIX_API_KEY"]
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

tracer_provider = register()

# Use Phoenix's autoinstrumentor to automatically track traces from Haystack
HaystackInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)

# Set up your Haystack app

In [ ]:
import os
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder

# Write documents to InMemoryDocumentStore
document_store = InMemoryDocumentStore()
document_store.write_documents([
    Document(content="My name is Jean and I live in Paris."),
    Document(content="My name is Mark and I live in Berlin."),
    Document(content="My name is Giorgio and I live in Rome.")
])

# Build a RAG pipeline
prompt_template = """
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""

retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_template)
llm = OpenAIGenerator(model="gpt-3.5-turbo")

rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [ ]:
# Ask a question
question = "Who lives in Paris?"
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

print(results["llm"]["replies"])

['Jean lives in Paris.']


# Evaluating Retrieved Docs

In [ ]:
import phoenix as px
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from phoenix.session.evaluation import get_retrieved_documents

client = px.Client()

retrieved_documents_df = get_retrieved_documents(px.Client())
retrieved_documents_df.head()

/usr/local/lib/python3.10/dist-packages/phoenix/utilities/client.py:25: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/phoenix/utilities/client.py:45: UserWarning: The Phoenix server (4.20.1) and client (4.24.0) versions are mismatched and may have compatibility issues.
  warnings.warn(


context.trace_id  \
context.span_id  document_position                                     
bba44d4539b972a9 0                  f7dd5d45af0f0c48c96d197d1763de52   
                 1                  f7dd5d45af0f0c48c96d197d1763de52   
                 2                  f7dd5d45af0f0c48c96d197d1763de52   

                                                                                input  \
context.span_id  document_position                                                      
bba44d4539b972a9 0                  {"query": "Who lives in Paris?", "filters": nu...   
                 1                  {"query": "Who lives in Paris?", "filters": nu...   
                 2                  {"query": "Who lives in Paris?", "filters": nu...   

                                                                 reference  \
context.span_id  document_position                                           
bba44d4539b972a9 0                    My name is Jean and I live in Paris.   
                 1                   My name is Mark and I live in Berlin.   
                 2                  My name is Giorgio and I live in Rome.   

                                    document_score  
context.span_id  document_position                  
bba44d4539b972a9 0                        1.293454  
                 1                        0.768010  
                 2                        0.768010

In [ ]:
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
    OpenAIModel
)

relevance_evaluator = RelevanceEvaluator(OpenAIModel(model="gpt-4o-mini"))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

run_evals |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

In [ ]:
retrieved_documents_relevance_df.head()

label  score  \
context.span_id  document_position                     
bba44d4539b972a9 0                   relevant      1   
                 1                  unrelated      0   
                 2                  unrelated      0   

                                                                          explanation  
context.span_id  document_position                                                     
bba44d4539b972a9 0                  The question asks who lives in Paris. The refe...  
                 1                  The question asks about who lives in Paris, wh...  
                 2                  The question asks about who lives in Paris, wh...

In [ ]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="relevance"),
)

/usr/local/lib/python3.10/dist-packages/phoenix/utilities/client.py:25: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/phoenix/utilities/client.py:45: UserWarning: The Phoenix server (4.20.1) and client (4.24.0) versions are mismatched and may have compatibility issues.
  warnings.warn(


# Evaluate Response

In [ ]:
from phoenix.session.evaluation import get_qa_with_reference

qa_with_reference_df = get_qa_with_reference(px.Client())
qa_with_reference_df

/usr/local/lib/python3.10/dist-packages/phoenix/utilities/client.py:25: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(


,input,output,reference
context.span_id,,,
16acfe8cec246618,"{""data"": {""retriever"": {""query"": ""Who lives in...","{""llm"": {""replies"": [""Jean lives in Paris.""], ...",My name is Jean and I live in Paris.\n\nMy nam...


In [ ]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    run_evals,
)

qa_evaluator = QAEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))
hallucination_evaluator = HallucinationEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))

qa_correctness_eval_df, hallucination_eval_df = run_evals(
    evaluators=[qa_evaluator, hallucination_evaluator],
    dataframe=qa_with_reference_df,
    provide_explanation=True,
    concurrency=20,
)

run_evals |          | 0/2 (0.0%) | ⏳ 00:00<? | ?it/s

In [ ]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=qa_correctness_eval_df, eval_name="Q&A Correctness"),
    SpanEvaluations(dataframe=hallucination_eval_df, eval_name="Hallucination"),
)

/usr/local/lib/python3.10/dist-packages/phoenix/utilities/client.py:25: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/phoenix/utilities/client.py:45: UserWarning: The Phoenix server (4.20.1) and client (4.27.0) versions are mismatched and may have compatibility issues.
  warnings.warn(
